# API


Author: Justin Chun-ting Ho

Date: 27 Nov 2023

Credit: Some sections are adopted from the slides prepared by Damian Trilling

### Beyond files

- we can write anything to files
- as long as we know the structure and encoding, we can unpack it into data
- we don't even need files!
- how about sending it directly through the internet?

### How does API work?

![API](https://voyager.postman.com/illustration/diagram-what-is-an-api-postman-illustration.svg)

### Example: Google Books API

You could try this in any browser: [https://www.googleapis.com/books/v1/volumes?q=isbn:9780261102217](https://www.googleapis.com/books/v1/volumes?q=isbn:9780261102217)

But how do we know how to use it? Read the [documentation](https://developers.google.com/books/docs/v1/using#PerformingSearch)!

In [ ]:
# A better way to do this

import json
from urllib.request import urlopen

api = "https://www.googleapis.com/books/v1/volumes?q="
query = "isbn:9780261102217"

# send a request and get a JSON response
resp = urlopen(api + query)
# parse JSON into Python as a dictionary
book_data = json.load(resp)

In [ ]:
volume_info = book_data["items"][0]["volumeInfo"]

print('Title: ' + volume_info['title'])
print('Author: ' + str(volume_info['authors']))
print('Publication Date: ' + volume_info['publishedDate'])

### Example: Youtube API

#### Getting an API key

- Go to [Google Cloud Platform](https://console.cloud.google.com/)

- Create a project in the Google Developers Console

- Enable YouTube Data API 

- Obtain your API key

#### Step by Step Guide

![](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*bCsi9C7yC8U-dVdWW4Zqhg.png)

![](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*k86eiqdHf9HhhxWKKnO7Sg.png)

![](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*DgLkgzXA9YkzMJC7Dh7JZg.png)

![](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*KzdLen4agoUi33_H0MutcA.png)

![](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*3HjyBix-P1gop_CPLYNpiQ.png)

![](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*rzq6FpRfV0ujb_B6nUoGEA.png)

![](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*FOTj3rvn0hGmHxgNz0x1Gw.png)

![](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*cLWdO9siuQE-3v0kPz-SAA.png)

Credit: [Pedro Hernández](https://medium.com/mcd-unison/youtube-data-api-v3-in-python-tutorial-with-examples-e829a25d2ebd)

#### Install google api package

- install the package with `conda install -c conda-forge google-api-python-client` or 
`pip install google-api-python-client`

### Simple video search

In [ ]:
# Setting Up
import googleapiclient.discovery
api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = "#################"
youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

### Getting a list of videos

In [ ]:
# The codes to send the request
request = youtube.search().list(
        part="id,snippet",
        type='video',
        q="Lord of the rings",
        maxResults=1
)
# Request execution
response = request.execute()
print(response)

In [ ]:
lotr_videos_ids = youtube.search().list(
        part="id",
        type='video',
        order="viewCount", # This can also be "date", "rating", "relevance" etc.
        q="Lord of the rings", # The search query
        maxResults=50,
        fields="items(id(videoId))"
).execute()

In [ ]:
lotr_videos_ids

In [ ]:
info = {
    'id':[],
    'title':[],
    'views':[]
}

for item in lotr_videos_ids['items']:
    vidId = item['id']['videoId']
    r = youtube.videos().list(
        part="statistics,snippet",
        id=vidId,
        fields="items(statistics),snippet(title)"
    ).execute()

    views = r['items'][0]['statistics']['viewCount']
    title = r['items'][0]['snippet']['title']
    info['id'].append(vidId)
    info['title'].append(title)
    info['views'].append(views)

df = pd.DataFrame(data=info)

### How to search by channel id?

First, you need to find the channel id, there are many tools for that, eg [this one](https://commentpicker.com/youtube-channel-id.php). While it is possible to search by username, sometimes it work, sometimes it doesn't.

Example: Last Week Tonight by John Oliva (https://www.youtube.com/@LastWeekTonight)

In [ ]:
# Some Channel Statistics
response = youtube.channels().list( 
    part='statistics', 
    id='UC3XTzVzaHQEd30rQbuvCtTQ').execute()

In [ ]:
# You can search by channel id, but it will not return everything
videos_ids = youtube.search().list(
        part="id",
        type='video',
        order="viewCount", # This can also be "date", "rating", "relevance" etc.
        channelId="UC3XTzVzaHQEd30rQbuvCtTQ", # The search query
        maxResults=500,
        fields="items(id(videoId))"
).execute()

In [ ]:
# A more robust way is to search by playlists. First, you need to get the playlists ids.
response = youtube.playlists().list( 
        part='contentDetails,snippet', 
        channelId='UC3XTzVzaHQEd30rQbuvCtTQ', 
        maxResults=50
    ).execute() 

playlists = []
for i in response['items']:
    playlists.append(i['id'])

In [ ]:
# Next, write a loop to search through all the pages

nextPageToken = None

while True: 

    response = youtube.playlistItems().list( 
        part='snippet', 
        playlistId=playlists[0], 
        maxResults=100, 
        pageToken=nextPageToken 
    ).execute() 

    # Iterate through all response and get video description 
    for item in response['items']: 
        description = item['snippet']['title']
        print(description) 
        print("\n")
    nextPageToken = response.get('nextPageToken') 
  
    if not nextPageToken: 
        break

### Exercise

Find the top 10 most viewed dogs and cats video on YouTube